In [1]:
# %load_ext lab_black

In [2]:
import numpy as np
import pandas as pd
import itertools as it

from random import normalvariate

import gc
import timeit
import sys
import os

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Revoking stored reported data
# https://ipython.org/ipython-doc/rel-0.12/config/extensions/storemagic.html

%store -r data_Averoy
%store -r data_Tingvoll
%store -r data_Surnadal
%store -r data_Sunndal
%store -r data_Vestnes
%store -r data_Laerdal

# Constructing hypothetical (synthetic) populations

In [3]:
# Create subdirectory /synthetic_data if it does not exist
path = './synthetic_data'

# Check whether directory already exists
if not os.path.exists(path):
  os.mkdir(path)
  print("Folder %s created!" % path)
else:
  print("Folder %s already exists" % path)

Folder ./synthetic_data already exists


### Calculate the initial sizes of all six population categories

In [4]:
def calculate_initial_size_of_each_category(population_data):
    total_number_calves = population_data[0] * population_data[3]
    total_number_adults = population_data[0] * (1 - population_data[3])
    total_number_stags = total_number_adults / (population_data[1] + 1)
    total_number_hinds = total_number_adults - total_number_stags

    female_calves_survivors = total_number_calves / 2
    male_calves_survivors = total_number_calves / 2

    young_hinds_survivors = total_number_hinds * population_data[4]
    adult_hinds_survivors = total_number_hinds - young_hinds_survivors

    young_stags_survivors = total_number_stags * population_data[5]
    adult_stags_survivors = total_number_stags - young_stags_survivors
    return (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )

### Calculate the winter survival of all six population categories
tested:
%%time
np.random.default_rng().normal(0.8, 0.04)
CPU times: user 270 µs, sys: 102 µs, total: 372 µs
Wall time: 356 µs

%%time
np.random.normal(0.8, 0.04)
CPU times: user 19 µs, sys: 9 µs, total: 28 µs
Wall time: 31 µs

%%time
normalvariate(0.8, 0.04)
CPU times: user 8 µs, sys: 1 µs, total: 9 µs
Wall time: 13.1 µs

When one samples only one normally distributed number, it seems that the built-in function normalvariate is dramatically faster than the new randomgenerator in numpy and more than twice as fast as the old numpy generator.
The built-in function is called by: 
from random import normalvariate

However, if one firste calls rng = np.random.default_rng(), and then
%%time
rng.normal(0.8, 0.04)
CPU times: user 19 µs, sys: 1e+03 ns, total: 20 µs
Wall time: 24.1 µs


Below the built-in function is used. The built-in is said to thread-safe: https://docs.python.org/3/library/random.html


In [5]:
def winter_mortality_all_categories(
    female_calves, young_hinds, adult_hinds, male_calves, young_stags, adult_stags
):
    # Assumed winter survival parameters for all six population categories
    w_survival = [
        normalvariate(0.8, 0.04),
        normalvariate(0.92, 0.03),
        normalvariate(0.93, 0.03),
        normalvariate(0.8, 0.04),
        normalvariate(0.87, 0.04),
        normalvariate(0.93, 0.03),
    ]

    if w_survival[0] >= 1.0:
        fraction_female_calves_surviving_winter = 1.0
    else:
        fraction_female_calves_surviving_winter = w_survival[0]

    if w_survival[1] >= 1.0:
        fraction_young_hinds_surviving_winter = 1.0
    else:
        fraction_young_hinds_surviving_winter = w_survival[1]

    if w_survival[2] >= 1.0:
        fraction_adult_hinds_surviving_winter = 1.0
    else:
        fraction_adult_hinds_surviving_winter = w_survival[2]

    if w_survival[3] >= 1.0:
        fraction_male_calves_surviving_winter = 1.0
    else:
        fraction_male_calves_surviving_winter = w_survival[3]

    if w_survival[4] >= 1.0:
        fraction_young_stags_surviving_winter = 1.0
    else:
        fraction_young_stags_surviving_winter = w_survival[4]

    if w_survival[5] >= 1.0:
        fraction_adult_stags_surviving_winter = 1.0
    else:
        fraction_adult_stags_surviving_winter = w_survival[5]

    female_calves_survivors = female_calves * fraction_female_calves_surviving_winter
    young_hinds_survivors = young_hinds * fraction_young_hinds_surviving_winter
    adult_hinds_survivors = adult_hinds * fraction_adult_hinds_surviving_winter

    male_calves_survivors = male_calves * fraction_male_calves_surviving_winter
    young_stags_survivors = young_stags * fraction_young_stags_surviving_winter
    adult_stags_survivors = adult_stags * fraction_adult_stags_surviving_winter

    return (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
        fraction_female_calves_surviving_winter,
        fraction_young_hinds_surviving_winter,
        fraction_adult_hinds_surviving_winter,
        fraction_male_calves_surviving_winter,
        fraction_young_stags_surviving_winter,
        fraction_adult_stags_surviving_winter,
    )

### Spring update of population categories 

In [6]:
def spring_update_of_categories(
    female_calves_survivors,
    young_hinds_survivors,
    adult_hinds_survivors,
    male_calves_survivors,
    young_stags_survivors,
    adult_stags_survivors,
):
    # Surviving female calves become one year old young hinds
    young_hinds = female_calves_survivors

    # Surviving young hinds born almost two years ago become adult hinds
    adult_hinds = young_hinds_survivors + adult_hinds_survivors

    # Surviving male calves become one year old young stags
    young_stags = male_calves_survivors

    # Surviving young stags born almost two years ago become adult stags
    adult_stags = young_stags_survivors + adult_stags_survivors
    return young_hinds, adult_hinds, young_stags, adult_stags

### Calculate number of female and male calves born in early summer

In [7]:
def early_summer_breeding(young_hinds_survivors, adult_hinds_survivors):

    # Assumed fertility parameters for young and adult hinds
    calves_per_young_hind = normalvariate(0.4, 0.05)

    calves_per_adult_hind = normalvariate(0.975, 0.008)
    if calves_per_adult_hind >= 1.0:
        calves_per_adult_hind = 1.0

    female_calves_born = (
        young_hinds_survivors * calves_per_young_hind / 2
        + adult_hinds_survivors * calves_per_adult_hind / 2
    )
    male_calves_born = female_calves_born
    return (
        female_calves_born,
        male_calves_born,
        calves_per_young_hind,
        calves_per_adult_hind,
    )

### Calculate putative emigration of stags

In [8]:
def emigration_of_stags(stags_per_hind_threshold, adult_stags, adult_hinds):
    """
    Does not account for the hypothesis that migration is primarily sported by
    the two years old stags. The way we deal with the emigration issue is highly
    idealized as we say that when the number of adult stags per adult hind is
    larger than a certain number, then stags will emigrate until the ratio falls
    below this number. The underlying hypothesis is that the stags are
    capable of making an assessment of their potential breeding success based on
    perceptual data during the winter and the spring and act on this information
    by use of a built in algorithm.
    """

    if stags_per_hind_threshold == 0 or adult_hinds < 1:
        number_of_migrants = 0
    else:
        number_of_migrants = adult_stags - stags_per_hind_threshold * adult_hinds
        if number_of_migrants <= 0:
            number_of_migrants = 0

    if number_of_migrants > 0:
        migration_percentage = 100 * number_of_migrants / adult_stags
    else:
        migration_percentage = 0

    adult_stags = adult_stags - number_of_migrants

    return adult_stags, number_of_migrants, migration_percentage

### Calculate number of individuals in each category surviving the hunting season

In [9]:
def transform_harvested_percentages_to_numbers(
    total_harvest,
    fraction_female_calves_harvested,
    fraction_male_calves_harvested,
    fraction_young_hinds_harvested,
    fraction_adult_hinds_harvested,
    fraction_young_stags_harvested,
    fraction_adult_stags_harvested,
):
    number_harvested_female_calves = np.round(
        np.multiply(np.array(fraction_female_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_hinds = np.round(
        np.multiply(np.array(fraction_young_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_hinds = np.round(
        np.multiply(np.array(fraction_adult_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_male_calves = np.round(
        np.multiply(np.array(fraction_male_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_stags = np.round(
        np.multiply(np.array(fraction_young_stags_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_stags = np.round(
        np.multiply(np.array(fraction_adult_stags_harvested), np.array(total_harvest))
        / 100
    )
    return (
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
    )

In [10]:
def survivors_after_hunting(
    number_harvested_female_calves,
    number_harvested_young_hinds,
    number_harvested_adult_hinds,
    number_harvested_male_calves,
    number_harvested_young_stags,
    number_harvested_adult_stags,
    female_calves_born,
    male_calves_born,
    young_hinds,
    adult_hinds,
    young_stags,
    adult_stags,
):
    female_calves_survivors = female_calves_born - number_harvested_female_calves
    young_hinds_survivors = young_hinds - number_harvested_young_hinds
    adult_hinds_survivors = adult_hinds - number_harvested_adult_hinds

    male_calves_survivors = male_calves_born - number_harvested_male_calves
    young_stags_survivors = young_stags - number_harvested_young_stags
    adult_stags_survivors = adult_stags - number_harvested_adult_stags

    if female_calves_survivors < 0:
        female_calves_survivors = 0
    if young_hinds_survivors < 0:
        young_hinds_survivors = 0
    if adult_hinds_survivors < 0:
        adult_hinds_survivors = 0

    if male_calves_survivors < 0:
        male_calves_survivors = 0
    if young_stags_survivors < 0:
        young_stags_survivors = 0
    if adult_stags_survivors < 0:
        adult_stags_survivors = 0
    return (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )

### Project population development from one year to the next

In [11]:
def project_population_development(
    number_harvested_female_calves,
    number_harvested_young_hinds,
    number_harvested_adult_hinds,
    number_harvested_male_calves,
    number_harvested_young_stags,
    number_harvested_adult_stags,
    female_calves_survivors,
    young_hinds_survivors,
    adult_hinds_survivors,
    male_calves_survivors,
    young_stags_survivors,
    adult_stags_survivors,
    stags_per_hind_threshold,
    year,
):

    # Let Mother winter take her toll
    (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
        fraction_female_calves_surviving_winter,
        fraction_young_hinds_surviving_winter,
        fraction_adult_hinds_surviving_winter,
        fraction_male_calves_surviving_winter,
        fraction_young_stags_surviving_winter,
        fraction_adult_stags_surviving_winter,
    ) = winter_mortality_all_categories(
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )

    # Let calves become young ones and young ones become adults
    young_hinds, adult_hinds, young_stags, adult_stags = spring_update_of_categories(
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )
    # Total population in the spring
    total_population_in_spring = young_hinds + adult_hinds + young_stags + adult_stags

    # Time for giving birth
    (
        female_calves_born,
        male_calves_born,
        calves_per_young_hind,
        calves_per_adult_hind,
    ) = early_summer_breeding(young_hinds_survivors, adult_hinds_survivors)

    # A tough decision made by stags
    adult_stags, number_of_migrants, migration_percentage = emigration_of_stags(
        stags_per_hind_threshold, adult_stags, adult_hinds
    )

    # Calculate ratio hinds per stag just before hunting season
    if (young_stags + adult_stags) > 1.0:
        hinds_per_stag_before_hunting = (young_hinds + adult_hinds) / (
            young_stags + adult_stags
        )
    else:
        hinds_per_stag_before_hunting = "NaN"

    # Total population just before hunting season starts
    total_population_before_hunting = (
        female_calves_born
        + young_hinds
        + adult_hinds
        + male_calves_born
        + young_stags
        + adult_stags
    )

    # Oh Deer, life is becoming dangerous
    (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    ) = survivors_after_hunting(
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
        female_calves_born,
        male_calves_born,
        young_hinds,
        adult_hinds,
        young_stags,
        adult_stags,
    )

    # Total population in December after end of hunting season
    total_population_after_hunting = (
        female_calves_survivors
        + young_hinds_survivors
        + adult_hinds_survivors
        + male_calves_survivors
        + young_stags_survivors
        + adult_stags_survivors
    )

    # Calculate ratio hinds per stag in December after end of hunting season
    if (young_stags_survivors + adult_stags_survivors) > 1:
        hinds_per_stag_after_hunting = (
            young_hinds_survivors + adult_hinds_survivors
        ) / (young_stags_survivors + adult_stags_survivors)
    else:
        hinds_per_stag_after_hunting = "NaN"

    return (
        total_population_in_spring,
        number_of_migrants,
        migration_percentage,
        hinds_per_stag_before_hunting,
        total_population_before_hunting,
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
        total_population_after_hunting,
        hinds_per_stag_after_hunting,
        fraction_female_calves_surviving_winter,
        fraction_young_hinds_surviving_winter,
        fraction_adult_hinds_surviving_winter,
        fraction_male_calves_surviving_winter,
        fraction_young_stags_surviving_winter,
        fraction_adult_stags_surviving_winter,
        calves_per_young_hind,
        calves_per_adult_hind,
    )

### Main script putting it all together

In [12]:
def run_the_show():

    # Setting up all initial conditions to be run
    # Make all combinations of the six initial conditions
    initial_population = np.arange(900, 4050, 50)
    initial_hinds_per_stag = np.arange(1.3, 2.1, 0.1)
    init_frac_calves = np.arange(0.2, 0.355, 0.01)
    init_frac_y_hinds = np.arange(0.09, 0.275, 0.01)  # of total number hinds
    init_frac_y_stags = np.arange(0.18, 0.545, 0.02)  # of total number stags
    stags_per_hind_threshold = np.array(
        [0, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    )  # not really initial condition, but initial premise
    all_initial_states = np.array(
        np.meshgrid(
            initial_population,
            initial_hinds_per_stag,
            stags_per_hind_threshold,
            init_frac_calves,
            init_frac_y_hinds,
            init_frac_y_stags,
        )
    ).T.reshape(-1, 6)

    # Using all six municipalties
    municipalities = ["Averoy", "Tingvoll", "Surnadal", "Sunndal", "Vestnes", "Laerdal"]
    data_municipality = [
        data_Averoy,
        data_Tingvoll,
        data_Surnadal,
        data_Sunndal,
        data_Vestnes,
        data_Laerdal,
    ]

    for q in range(len(municipalities)):
        [
            municipality,
            first_year,
            last_year,
            number_of_years,
            years,
            seen_deer_obs,
            seen_deer_obs_outfield,
            seen_deer_obs_infield,
            hinds_per_stag_obs,
            total_harvest,
            fraction_female_calves_harvested,
            fraction_young_hinds_harvested,
            fraction_adult_hinds_harvested,
            fraction_male_calves_harvested,
            fraction_young_stags_harvested,
            fraction_adult_stags_harvested,
            spring_counts,
        ] = data_municipality[q]

        # Transform reported hunting percentages for all years to numbers
        (
            number_harvested_female_calves,
            number_harvested_young_hinds,
            number_harvested_adult_hinds,
            number_harvested_male_calves,
            number_harvested_young_stags,
            number_harvested_adult_stags,
        ) = transform_harvested_percentages_to_numbers(
            total_harvest,
            fraction_female_calves_harvested,
            fraction_male_calves_harvested,
            fraction_young_hinds_harvested,
            fraction_adult_hinds_harvested,
            fraction_young_stags_harvested,
            fraction_adult_stags_harvested,
        )

        # Initiating the total log array for a given population
        population_log = np.zeros(
            (
                len(all_initial_states) * number_of_years,
                31,
            )
        )

        counter = 0
        for state in all_initial_states:
            # Calculate size of each category in the initial population December 31 year -1
            (
                female_calves_survivors,
                young_hinds_survivors,
                adult_hinds_survivors,
                male_calves_survivors,
                young_stags_survivors,
                adult_stags_survivors,
            ) = calculate_initial_size_of_each_category(state)

            # Record initial sizes of the six categories
            female_calves_initially = female_calves_survivors
            young_hinds_initially = young_hinds_survivors
            adult_hinds_initially = adult_hinds_survivors
            male_calves_initially = male_calves_survivors
            young_stags_initially = young_stags_survivors
            adult_stags_initially = adult_stags_survivors

            # Run through all years in the time series for each initial state
            for j in range(number_of_years):
                (
                    total_population_in_spring,
                    number_of_migrants,
                    migration_percentage,
                    hinds_per_stag_before_hunting,
                    total_population_before_hunting,
                    female_calves_survivors,
                    young_hinds_survivors,
                    adult_hinds_survivors,
                    male_calves_survivors,
                    young_stags_survivors,
                    adult_stags_survivors,
                    total_population_after_hunting,
                    hinds_per_stag_after_hunting,
                    fraction_female_calves_surviving_winter,
                    fraction_young_hinds_surviving_winter,
                    fraction_adult_hinds_surviving_winter,
                    fraction_male_calves_surviving_winter,
                    fraction_young_stags_surviving_winter,
                    fraction_adult_stags_surviving_winter,
                    calves_per_young_hind,
                    calves_per_adult_hind,
                ) = project_population_development(
                    number_harvested_female_calves[j],
                    number_harvested_young_hinds[j],
                    number_harvested_adult_hinds[j],
                    number_harvested_male_calves[j],
                    number_harvested_young_stags[j],
                    number_harvested_adult_stags[j],
                    female_calves_survivors,
                    young_hinds_survivors,
                    adult_hinds_survivors,
                    male_calves_survivors,
                    young_stags_survivors,
                    adult_stags_survivors,
                    state[2],
                    j,
                )

                population_log[counter, :] = [
                    state[0],
                    state[1],
                    state[2],
                    female_calves_initially,
                    young_hinds_initially,
                    adult_hinds_initially,
                    male_calves_initially,
                    young_stags_initially,
                    adult_stags_initially,
                    first_year + j,
                    female_calves_survivors,
                    young_hinds_survivors,
                    adult_hinds_survivors,
                    male_calves_survivors,
                    young_stags_survivors,
                    adult_stags_survivors,
                    total_population_in_spring,
                    total_population_before_hunting,
                    total_population_after_hunting,
                    hinds_per_stag_before_hunting,
                    hinds_per_stag_after_hunting,
                    migration_percentage,
                    number_of_migrants,
                    fraction_female_calves_surviving_winter,
                    fraction_young_hinds_surviving_winter,
                    fraction_adult_hinds_surviving_winter,
                    fraction_male_calves_surviving_winter,
                    fraction_young_stags_surviving_winter,
                    fraction_adult_stags_surviving_winter,
                    calves_per_young_hind,
                    calves_per_adult_hind,
                ]

                counter = counter + 1

        # Make pandas dataframe for given municipality
        column_values = [
            "init_pop",
            "init_hps",
            "sh_threshold",
            "i_fcalves",
            "i_yhinds",
            "i_ahinds",
            "i_mcalves",
            "i_ystags",
            "i_astags",
            "obs_year",
            "f_calves",
            "y_hinds",
            "a_hinds",
            "m_calves",
            "y_stags",
            "a_stags",
            "spring_pop",
            "tot_pop_bh",
            "tot_pop_ah",
            "hps_bh",
            "hps_ah",
            "mig%",
            "num_migs",
            "ws_fc",
            "ws_yh",
            "ws_ah",
            "ws_mc",
            "ws_ys",
            "ws_as",
            "c_yh",
            "c_ah",
        ]
        pop_data = pd.DataFrame(data=population_log, columns=column_values)
        pop_data = pop_data.astype(
            {
                "init_pop": "int",
                "obs_year": "int",
                "i_fcalves": "int",
                "i_yhinds": "int",
                "i_ahinds": "int",
                "i_mcalves": "int",
                "i_ystags": "int",
                "i_astags": "int",
                "f_calves": "int",
                "y_hinds": "int",
                "a_hinds": "int",
                "m_calves": "int",
                "y_stags": "int",
                "a_stags": "int",
                "spring_pop": "int",
                "tot_pop_bh": "int",
                "tot_pop_ah": "int",
                "num_migs": "int",
            }
        ).round(4)

        # Sort the data frame
        pop_data_sorted = pop_data.sort_values(
            by=["init_pop", "init_hps", "sh_threshold"]  # , "obs_year"
        ).reset_index(drop=True)

        # Save the sorted frame
        save_dir = "synthetic_data/"
        pop_data_sorted.to_pickle(
            save_dir + "df_original_sorted_" + municipality + ".pkl"
        )

        print("Municipality = ", municipality)
        display(pop_data_sorted)
        print()
        print()

In [13]:
%%time
# Running all scripts by calling up run_the_show
# This is done to get rid of memory leaks
run_the_show()

Municipality =  Averoy


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,obs_year,...,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah
0,900,1.3,0.0,90,36,370,90,56,256,2007,...,0.0,0,0.8146,0.9176,0.8836,0.7461,0.8701,0.9246,0.4891,0.9774
1,900,1.3,0.0,90,36,370,90,56,256,2008,...,0.0,0,0.7957,0.9194,0.9149,0.8186,0.8475,0.9304,0.4167,0.9726
2,900,1.3,0.0,90,36,370,90,56,256,2009,...,0.0,0,0.8040,0.9287,0.9094,0.8211,0.8639,0.9404,0.4417,0.9915
3,900,1.3,0.0,90,36,370,90,56,256,2010,...,0.0,0,0.8869,0.9290,0.9509,0.7932,0.9385,0.9681,0.4647,0.9738
4,900,1.3,0.0,90,36,370,90,56,256,2011,...,0.0,0,0.8055,0.9190,0.8947,0.7664,0.8661,0.9489,0.3899,0.9640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305665915,4000,2.0,1.0,700,467,1265,700,467,398,2017,...,0.0,0,0.8637,0.9151,0.9493,0.8394,0.8633,0.9508,0.3386,0.9570
305665916,4000,2.0,1.0,700,467,1265,700,467,398,2018,...,0.0,0,0.7900,0.9337,0.9360,0.8302,0.9267,0.9397,0.3657,0.9703
305665917,4000,2.0,1.0,700,467,1265,700,467,398,2019,...,0.0,0,0.7754,0.9075,0.9751,0.8298,0.8967,0.9479,0.4760,0.9746
305665918,4000,2.0,1.0,700,467,1265,700,467,398,2020,...,0.0,0,0.7210,0.9591,0.9122,0.7698,0.8374,0.9164,0.3811,0.9560




Municipality =  Tingvoll


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,obs_year,...,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah
0,900,1.3,0.0,90,36,370,90,56,256,2007,...,0.0000,0,0.8029,0.9329,0.9535,0.8260,0.8336,0.9650,0.3906,0.9760
1,900,1.3,0.0,90,36,370,90,56,256,2008,...,0.0000,0,0.8249,0.9457,0.9139,0.7611,0.9076,0.9459,0.4441,0.9769
2,900,1.3,0.0,90,36,370,90,56,256,2009,...,0.0000,0,0.7699,0.9048,0.9175,0.8233,0.8616,0.9603,0.4159,0.9580
3,900,1.3,0.0,90,36,370,90,56,256,2010,...,0.0000,0,0.8259,0.9125,0.9309,0.8612,0.8472,0.9207,0.4592,0.9766
4,900,1.3,0.0,90,36,370,90,56,256,2011,...,0.0000,0,0.7825,0.9736,0.8820,0.8535,0.8989,0.9399,0.4265,0.9674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305665915,4000,2.0,1.0,700,467,1265,700,467,398,2017,...,0.0000,0,0.8046,0.9479,0.9331,0.8505,0.8272,0.9436,0.2918,0.9793
305665916,4000,2.0,1.0,700,467,1265,700,467,398,2018,...,0.0000,0,0.8617,0.9151,0.9148,0.7464,0.8923,0.8981,0.4587,0.9845
305665917,4000,2.0,1.0,700,467,1265,700,467,398,2019,...,2.0948,107,0.8049,0.8708,0.8799,0.7592,0.9488,0.9278,0.3719,0.9826
305665918,4000,2.0,1.0,700,467,1265,700,467,398,2020,...,1.9732,115,0.8413,0.9062,0.9108,0.7835,0.8854,0.9404,0.4176,0.9799




Municipality =  Surnadal


...



Municipality =  Vestnes


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,obs_year,...,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah
0,900,1.3,0.0,90,36,370,90,56,256,2007,...,0.0,0,0.8043,0.9552,0.9202,0.7751,0.9211,0.9133,0.4299,0.9832
1,900,1.3,0.0,90,36,370,90,56,256,2008,...,0.0,0,0.8000,0.9296,0.9828,0.8782,0.8743,0.9433,0.4010,0.9853
2,900,1.3,0.0,90,36,370,90,56,256,2009,...,0.0,0,0.8625,0.9169,0.9087,0.7568,0.9331,0.8812,0.5077,0.9760
3,900,1.3,0.0,90,36,370,90,56,256,2010,...,0.0,0,0.7890,0.9349,0.9117,0.7379,0.9012,0.9320,0.4133,0.9682
4,900,1.3,0.0,90,36,370,90,56,256,2011,...,0.0,0,0.8454,0.9252,0.9287,0.8477,0.8664,0.9656,0.4621,0.9742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305665915,4000,2.0,1.0,700,467,1265,700,467,398,2017,...,0.0,0,0.8467,0.9732,0.9146,0.8093,0.7846,0.9654,0.3748,0.9802
305665916,4000,2.0,1.0,700,467,1265,700,467,398,2018,...,0.0,0,0.7421,0.8964,0.9479,0.8331,0.9234,0.9399,0.4364,0.9821
305665917,4000,2.0,1.0,700,467,1265,700,467,398,2019,...,0.0,0,0.8170,0.9217,0.9431,0.7975,0.8080,0.8974,0.5438,0.9800
305665918,4000,2.0,1.0,700,467,1265,700,467,398,2020,...,0.0,0,0.7441,0.9200,0.9620,0.8337,0.8173,0.9463,0.3201,0.9730




Municipality =  Laerdal


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,obs_year,...,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah
0,900,1.3,0.0,90,36,370,90,56,256,2007,...,0.00,0,0.8424,0.9062,0.9076,0.7968,0.9259,0.8899,0.3762,0.9795
1,900,1.3,0.0,90,36,370,90,56,256,2008,...,0.00,0,0.7978,0.8938,0.9306,0.7721,0.8645,0.8909,0.4002,0.9581
2,900,1.3,0.0,90,36,370,90,56,256,2009,...,0.00,0,0.7790,0.9251,0.9276,0.8351,0.8486,0.9326,0.4440,0.9857
3,900,1.3,0.0,90,36,370,90,56,256,2010,...,0.00,0,0.7679,0.9222,0.9469,0.8057,0.7877,0.9184,0.3734,0.9816
4,900,1.3,0.0,90,36,370,90,56,256,2011,...,0.00,0,0.8057,0.9263,0.9348,0.8176,0.8993,0.9234,0.4216,0.9706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305665915,4000,2.0,1.0,700,467,1265,700,467,398,2017,...,0.52,31,0.8466,0.9058,0.9723,0.7780,0.8770,1.0000,0.4882,0.9785
305665916,4000,2.0,1.0,700,467,1265,700,467,398,2018,...,0.00,0,0.8490,0.9820,0.9395,0.8052,0.7712,0.9326,0.4176,0.9712
305665917,4000,2.0,1.0,700,467,1265,700,467,398,2019,...,0.00,0,0.8048,0.9516,0.9138,0.8334,0.9293,0.9358,0.4151,0.9729
305665918,4000,2.0,1.0,700,467,1265,700,467,398,2020,...,0.00,0,0.7143,0.8924,0.9593,0.7433,0.8872,0.9264,0.3216,0.9765




CPU times: user 6h 16min 42s, sys: 23min 25s, total: 6h 40min 8s
Wall time: 6h 42min 45s
